In [3]:
from jinja2 import Template

from copy import deepcopy

oil_template = Template("""
OIL_VERSION = "2.5" : "test" ;

CPU test {
  OS config {
    STATUS = STANDARD;
    BUILD = TRUE {
      TRAMPOLINE_BASE_PATH = "../trampoline";
      APP_NAME = "avr-AUTOSAR-trampoline-Blink.elf";
      APP_SRC = "avr-AUTOSAR-trampoline-Blink.cpp";
      CPPCOMPILER = "avr-g++";
      COMPILER = "avr-gcc";
      LINKER = "avr-gcc";
      ASSEMBLER = "avr-gcc";
      COPIER = "avr-objcopy";
      SYSTEM = PYTHON;
      LIBRARY = serial;
    };
    SYSTEM_CALL = TRUE;
  };

  APPMODE stdAppmode {};

{% for task in tasks %}
  ALARM {{ task.name }}Alarm {
    COUNTER= SystemCounter;
    ACTION = ACTIVATETASK {
      TASK = {{ task.name }};
    };
    AUTOSTART = TRUE {
      ALARMTIME = {{ task.alarmtime }};
      CYCLETIME = {{ task.cycletime }};
      APPMODE = stdAppmode;
    };
  };
  TASK {{ task.name }} {
    PRIORITY = {{ task.alarmtime }};
    AUTOSTART = FALSE;
    ACTIVATION = 1;
    SCHEDULE = FULL;
    STACKSIZE = {{ task.stacksize }};
  };
{% endfor %}

};
""")

In [9]:
tasks = list()
task_default = {
    "alarmtime": 5000,
    "cycletime": 10,
    "priority": 20,
    "stacksize": 256,
}

cycletime=10
for pin in range(8, 14):
    task = deepcopy(task_default)
    task["name"]=f"blinkTaskPin{pin}"
    task["cycletime"]=cycletime
    tasks.append(task)
    cycletime=cycletime*2
with open("avr-AUTOSAR-trampoline-Blink.oil", "w") as fid:
    print(oil_template.render(tasks=tasks), file=fid)

In [8]:
oil_template.render?